In [1]:
import cv2 as cv
import numpy as np
import copy
import os
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
def diff(image_1, image_2):
    image_1 = np.float32(image_1)  
    image_2 = np.float32(image_2)  
    return np.mean((image_1 - image_2) ** 2)

In [3]:
def show_image(image_, window_name='image', timeout=0):
    cv.imshow(window_name, np.uint8(image_))
    cv.waitKey(timeout)
    cv.destroyAllWindows()

In [4]:
def load_vh_tiles():
    vertical_tiles = []
    horizontal_tiles = []
    for i in range(7):
        v = cv.imread("templates/tiles/vertical/" + str(i) + ".jpg", cv.IMREAD_GRAYSCALE)
        _, v_bin = cv.threshold(v, 150, 255, cv.THRESH_BINARY)
        vertical_tiles.append(v_bin)
        h = cv.imread("templates/tiles/horizontal/" + str(i) + ".jpg", cv.IMREAD_GRAYSCALE)
        _, h_bin = cv.threshold(h, 150, 255, cv.THRESH_BINARY)
        horizontal_tiles.append(h_bin)
    return vertical_tiles, horizontal_tiles

In [5]:
def get_keypoints_and_features(image, show_details=False) -> tuple:  
    def show_keypoints(image_, keypoints_):
        image_output = image_.copy()
        image_output = cv.drawKeypoints(image, keypoints_, image_output, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        show_image(image_output, 'keypoints')
        
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    sift = cv.SIFT_create()
    
    keypoints, features = sift.detectAndCompute(gray_image, None)
     
    if show_details:
        show_keypoints(image, keypoints)
        
    return keypoints, features

In [6]:
def match_features(features_source, features_dest) -> [[cv.DMatch]]: 
    feature_matcher = cv.DescriptorMatcher_create("FlannBased")
    matches = feature_matcher.knnMatch(features_source, features_dest, k=2)   
    return matches

In [7]:
def generate_homography(all_matches:  [cv.DMatch], keypoints_source: [cv.KeyPoint], keypoints_dest : [cv.KeyPoint],
                        ratio: float = 0.75, ransac_rep: int = 4.0):
    if not all_matches:
        return None
    
    matches = []
    for match in all_matches:
        if len(match) == 2 and (match[0].distance/match[1].distance) < ratio:
            matches.append(match[0])
    points_source = np.float32([keypoints_source[m.queryIdx].pt for m in matches])
    points_destination = np.float32([keypoints_dest[m.trainIdx].pt for m in matches])
    
    if len(points_source) > 4:
        H, status = cv.findHomography(points_source, points_destination, cv.RANSAC, ransac_rep)
        return H
    else:
        return None

In [8]:
def stitch_images(image_source, image_dest, show_details=False):
    def show_matches(all_matches_, n=10):
        matches = sorted(all_matches_, key = lambda x:x[0].distance)
        matches = matches[:n] 
        image_output = cv.drawMatchesKnn(image_source, keypoints_source, 
                                         image_dest, keypoints_dest,  
                                         matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        show_image(image_output, 'matches')
    
    keypoints_source, features_source = get_keypoints_and_features(image_source, show_details)
    keypoints_dest, features_dest = get_keypoints_and_features(image_dest, show_details)
    
    all_matches = match_features(features_source, features_dest)
    
    if show_details:
        show_matches(copy.copy(all_matches))
        
    H = generate_homography(all_matches, keypoints_source, keypoints_dest)
    
    result = cv.warpPerspective(image_source, H, (image_dest.shape[1], image_dest.shape[0]))
    return result

In [9]:
template = cv.imread("templates/board/1.jpg")
col_no_to_ch = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J', 10:'K', 11:'L', 12:'M', 13:'N', 14:'O'}
rows = [116, 154, 191, 228, 265, 303, 340, 377, 415, 452, 490, 528, 566, 604, 642, 681]
cols = [122, 161, 199, 236, 274, 311, 348, 385, 422, 458, 495, 533, 569, 607, 643, 682]
vertical_tiles, horizontal_tiles = load_vh_tiles()
diamonds = [[5, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5],
            [0, 0, 3, 0, 0, 4, 0, 0, 0, 4, 0, 0, 3, 0, 0],
            [0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0],
            [4, 0, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 4],
            [0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
            [0, 4, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 4, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
            [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
            [0, 4, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 4, 0],
            [0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
            [4, 0, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 4],
            [0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0],
            [0, 0, 3, 0, 0, 4, 0, 0, 0, 4, 0, 0, 3, 0, 0],
            [5, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5]]
track = [1, 2, 3, 4, 5, 6, 0, 2, 5, 3, 4, 6, 2, 2, 0, 3, 5, 4, 1, 6, 2, 4, 5, 5, 0,
         6, 3, 4, 2, 0, 1, 5, 1, 3, 4, 4, 4, 5, 0, 6, 3, 5, 4, 1, 3, 2, 0, 0, 1, 1,
         2, 3, 6, 3, 5, 2, 1, 0, 6, 6, 5, 2, 1, 2, 5, 0, 3, 3, 5, 0, 6, 1, 4, 0, 6,
         3, 5, 1, 4, 2, 6, 2, 3, 1, 6, 5, 6, 2, 0, 4, 0, 1, 6, 4, 4, 1, 6, 6, 3, 0]

In [10]:
def classify_tile(img):
    vertical_scores = []
    horizontal_scores = []
    show_image(img)
    for t in vertical_tiles:
        result = cv.matchTemplate(img, t, cv.TM_SQDIFF_NORMED)
        min_val, max_val, _, _ = cv.minMaxLoc(result)
        vertical_scores.append(min_val)
    for t in horizontal_tiles:
        result = cv.matchTemplate(img, t, cv.TM_SQDIFF_NORMED)
        min_val, max_val, _, _ = cv.minMaxLoc(result)
        horizontal_scores.append(min_val)
    if min(vertical_scores) < min(horizontal_scores):
        return np.argmin(vertical_scores)
    else:
        return np.argmin(horizontal_scores)

In [11]:
def is_inside(i, j):
    if i < 0 or j < 0 or i > 14 or j > 14:
        return False
    return True

In [12]:
def has_neighbour(i, j, g):
    if is_inside(i-1, j) and g[i-1][j] is True:
        return True
    if is_inside(i+1, j) and g[i+1][j] is True:
        return True
    if is_inside(i, j-1) and g[i][j-1] is True:
        return True
    if is_inside(i, j+1) and g[i][j+1] is True:
        return True
    return False

In [13]:
def write_result_regular(move_no, i1, j1, i2, j2, patch1_tile, patch2_tile, move_score):
    res = ""
    poz1 = str(i1+1) + col_no_to_ch[j1] + " " + str(patch1_tile)
    poz2 = str(i2+1) + col_no_to_ch[j2] + " " + str(patch2_tile)
    if i1 == i2:
        if j1 < j2:
            res += poz1 + "\n" + poz2
        else:
            res += poz2 + "\n" + poz1
    elif i1 < i2:
        res += poz1 + "\n" + poz2
    else:
        res += poz2 + "\n" + poz1
    res += "\n" + str(move_score)
    f = open("results/regular_tasks/" + move_no + ".txt", "w")
    f.write(res)
    f.close()

In [14]:
def play_game(game_no):
    dir_path = "test/regular_tasks/"
    players = {}
    f_moves = open(dir_path + str(game_no)+'_moves.txt')
    for _ in range(20):
        parsed_line = f_moves.readline().strip().split()
        players[parsed_line[0]] = parsed_line[1]
    f_moves.close()
    player1_full_score = 0
    player2_full_score = 0
    prev = copy.copy(template)
    game = [[False for _ in range(15)] for _ in range(15)]
    game[7][7] = True
    i1, j1 = 7, 7
    for im in range(1, 21):
        str_im = ('0' + str(im)) if im < 10 else str(im)
        move_no = str(game_no) + '_' + str_im
        img_path = move_no + '.jpg'
        img = cv.imread(dir_path + img_path)
        img = cv.resize(img, None, fx=0.3, fy=0.3) 
        wb = stitch_images(img, template, False)
        move_score = 0
        
        #show_image(wb)
        
        if im > 1:
            i1, j1, max1 = None, None, -np.inf
            for i in range(15):
                for j in range(15):
                    if game[i][j] is False and has_neighbour(i, j, game):
                        difference = diff(prev[rows[i]+5:rows[i+1]-5, cols[j]+5:cols[j+1]-5], \
                                          wb[rows[i]+5:rows[i+1]-5, cols[j]+5:cols[j+1]-5])
                        if difference > max1:
                            max1 = difference
                            i1 = i
                            j1 = j
            game[i1][j1] = True
                            
        i2, j2, max2 = None, None, -np.inf
        
        if is_inside(i1-1, j1) and game[i1-1][j1] is False:
            difference = diff(prev[rows[i1-1]+5:rows[i1]-5, cols[j1]+5:cols[j1+1]-5], \
                              wb[rows[i1-1]+5:rows[i1]-5, cols[j1]+5:cols[j1+1]-5])
            if difference > max2:
                max2 = difference
                i2 = i1-1
                j2 = j1
        if is_inside(i1+1, j1) and game[i1+1][j1] is False:
            difference = diff(prev[rows[i1+1]+5:rows[i1+2]-5, cols[j1]+5:cols[j1+1]-5], \
                              wb[rows[i1+1]+5:rows[i1+2]-5, cols[j1]+5:cols[j1+1]-5])
            if difference > max2:
                max2 = difference
                i2 = i1+1
                j2 = j1
        if is_inside(i1, j1-1) and game[i1][j1-1] is False:
            difference = diff(prev[rows[i1]+5:rows[i1+1]-5, cols[j1-1]+5:cols[j1]-5], \
                              wb[rows[i1]+5:rows[i1+1]-5, cols[j1-1]+5:cols[j1]-5])
            if difference > max2:
                max2 = difference
                i2 = i1
                j2 = j1-1
        if is_inside(i1, j1+1) and game[i1][j1+1] is False:
            difference = diff(prev[rows[i1]+5:rows[i1+1]-5, cols[j1+1]+5:cols[j1+2]-5], \
                              wb[rows[i1]+5:rows[i1+1]-5, cols[j1+1]+5:cols[j1+2]-5])
            if difference > max2:
                max2 = difference
                i2 = i1
                j2 = j1+1

        game[i2][j2] = True
        prev = copy.copy(wb)
        
        gray_board = cv.cvtColor(wb, cv.COLOR_BGR2GRAY)
        patch1 = gray_board[rows[i1]-5:rows[i1+1]+5, cols[j1]-5:cols[j1+1]+5]
        patch2 = gray_board[rows[i2]-5:rows[i2+1]+5, cols[j2]-5:cols[j2+1]+5]
        _, patch1_bin = cv.threshold(patch1, 150, 255, cv.THRESH_BINARY)
        _, patch2_bin = cv.threshold(patch2, 150, 255, cv.THRESH_BINARY)
        patch1_tile = classify_tile(patch1_bin)
        patch2_tile = classify_tile(patch2_bin)
        
        move_score = move_score + diamonds[i1][j1] + diamonds[i2][j2]
        if patch1_tile == patch2_tile:
            move_score = move_score + diamonds[i1][j1] + diamonds[i2][j2]
        if players[img_path] == 'player1':
            if player1_full_score > 0 and (track[player1_full_score-1] in [patch1_tile, patch2_tile]):
                move_score += 3
            if player2_full_score > 0 and (track[player2_full_score-1] in [patch1_tile, patch2_tile]):
                player2_full_score += 3
            player1_full_score += move_score
        elif players[img_path] == 'player2':
            if player2_full_score > 0 and (track[player2_full_score-1] in [patch1_tile, patch2_tile]):
                move_score += 3
            if player1_full_score > 0 and (track[player1_full_score-1] in [patch1_tile, patch2_tile]):
                player1_full_score += 3
            player2_full_score += move_score
                
        write_result_regular(move_no, i1, j1, i2, j2, patch1_tile, patch2_tile, move_score)
#         show_image(patch1_bin)
#         show_image(patch2_bin)
#         print(str(i1+1) + col_no_to_ch[j1], patch1_tile, \
#               str(i2+1) + col_no_to_ch[j2], patch2_tile, '    ', move_score)
                  
                        

In [15]:
#play_game(5)

In [15]:
def regular_task(no_games):
    res_path = 'results/'
    if not os.path.exists(res_path):
        os.mkdir(res_path)
    regular_path = 'results/regular_tasks/'
    if not os.path.exists(regular_path):
        os.mkdir(regular_path)
    for i in range(1, no_games + 1):
        play_game(i)
        print("Finished game " + str(i))

In [16]:
def empty_neighbours(i, j, b):
    neighbours = 0
    if (not is_inside(i-1, j)) or (is_inside(i-1, j) and b[i-1][j] == -1):
        neighbours += 1
    if (not is_inside(i+1, j)) or (is_inside(i+1, j) and b[i+1][j] == -1):
        neighbours += 1
    if (not is_inside(i, j-1)) or (is_inside(i, j-1) and b[i][j-1] == -1):
        neighbours += 1
    if (not is_inside(i, j+1)) or (is_inside(i, j+1) and b[i][j+1] == -1):
        neighbours += 1
    return neighbours

In [17]:
def pieces_neighbours(i, j, b):
    neighbours = 0
    if is_inside(i-1, j) and b[i-1][j] > 0:
        neighbours += 1
    if is_inside(i+1, j) and b[i+1][j] > 0:
        neighbours += 1
    if is_inside(i, j-1) and b[i][j-1] > 0:
        neighbours += 1
    if is_inside(i, j+1) and b[i][j+1] > 0:
        neighbours += 1
    return neighbours

In [18]:
def get_other_half(i, j, b):
    if is_inside(i-1, j) and b[i-1][j] == 0:
        return i-1, j
    if is_inside(i+1, j) and b[i+1][j] == 0:
        return i+1, j
    if is_inside(i, j-1) and b[i][j-1] == 0:
        return i, j-1
    if is_inside(i, j+1) and b[i][j+1] == 0:
        return i, j+1
    return i, j

In [19]:
def get_pieces(config):
    pieces = [[-1 for _ in range(15)] for _ in range(15)]
    for i in range(15):
        for j in range(15):
            if config[i][j] is not None:
                pieces[i][j] = 0
    
    piece_no = 1
    
    found = True
    while found:
        found = False
        for i in range(15):
            for j in range(15):
                if pieces[i][j] == 0 and (empty_neighbours(i, j, pieces) == 3 or \
                                         (empty_neighbours(i, j, pieces) == 2 and pieces_neighbours(i, j, pieces) == 1) or \
                                         (empty_neighbours(i, j, pieces) == 1 and pieces_neighbours(i, j, pieces) == 2) or \
                                         pieces_neighbours(i, j, pieces) == 3):
                    found = True
                    pieces[i][j] = piece_no
                    i_half, j_half = get_other_half(i, j, pieces)
                    pieces[i_half][j_half] = piece_no
                    piece_no += 1
                    
    return pieces
        

In [20]:
def write_result_bonus(img_no, mistakes_no, mistakes):
    res = str(mistakes_no) + "\n"
    for i in range(15):
        for j in range(15):
            if mistakes[i][j] == True:
                res += str(i+1) + col_no_to_ch[j] + "\n"
    res = res[:-1]
    str_im = ('0' + str(img_no)) if img_no < 10 else str(img_no)
    f = open("results/bonus_task/" + str_im + ".txt", "w")
    f.write(res)
    f.close()

In [21]:
def find_mistakes(board_no):
    dir_path = "test/bonus_task/"
    str_im = ('0' + str(board_no)) if board_no < 10 else str(board_no)
    img_path = str_im + ".jpg"
    img = cv.imread(dir_path + img_path)
    img = cv.resize(img, None, fx=0.3, fy=0.3) 
    wb = stitch_images(img, template, False)
    gray_board = cv.cvtColor(wb, cv.COLOR_BGR2GRAY)
    difs = [[0 for _ in range(15)] for _ in range(15)]
    bd = [[None for _ in range(15)] for _ in range(15)]

    for i in range(15):
        for j in range(15):
            difs[i][j] = diff(template[rows[i]+5:rows[i+1]-5, cols[j]+5:cols[j+1]-5], wb[rows[i]+5:rows[i+1]-5, cols[j]+5:cols[j+1]-5])
            if diff(template[rows[i]+5:rows[i+1]-5, cols[j]+5:cols[j+1]-5], wb[rows[i]+5:rows[i+1]-5, cols[j]+5:cols[j+1]-5]) > 4750:
                patch = gray_board[rows[i]-5:rows[i+1]+5, cols[j]-5:cols[j+1]+5]
                _, patch_bin = cv.threshold(patch, 150, 255, cv.THRESH_BINARY)
                bd[i][j] = classify_tile(patch_bin)
                
    pieces = get_pieces(bd)
    
    mistakes = [[False for _ in range(15)] for _ in range(15)]
    mistakes_no = 0
    
    for i in range(14):
        for j in range(14):
            if bd[i][j] is not None and bd[i+1][j] is not None and bd[i][j+1] is not None and bd[i+1][j+1] is not None:
                mistakes_no += 1
                mistakes[i][j] = True
                mistakes[i+1][j] = True
                mistakes[i][j+1] = True
                mistakes[i+1][j+1] = True
                
    for i in range(15):
        for j in range(14):
            if bd[i][j] is not None and bd[i][j+1] is not None and bd[i][j] != bd[i][j+1] and pieces[i][j] != pieces[i][j+1]:
                mistakes_no += 1
                mistakes[i][j] = True
                mistakes[i][j+1] = True
                
    for i in range(14):
        for j in range(15):
            if bd[i][j] is not None and bd[i+1][j] is not None and bd[i][j] != bd[i+1][j] and pieces[i][j] != pieces[i+1][j]:
                mistakes_no += 1
                mistakes[i][j] = True
                mistakes[i+1][j] = True
                
    #write_result_bonus(board_no, mistakes_no, mistakes)
                

In [31]:
find_mistakes(10)

In [24]:
def bonus_task(no_img):
    res_path = 'results/'
    if not os.path.exists(res_path):
        os.mkdir(res_path)
    bonus_path = 'results/bonus_task/'
    if not os.path.exists(bonus_path):
        os.mkdir(bonus_path)
    for im in range(1, no_img+1):
        find_mistakes(im) 
        print("Finished image " + str(im))

In [25]:
regular_task(5)

Finished game 1
Finished game 2
Finished game 3
Finished game 4
Finished game 5


In [26]:
bonus_task(10)

Finished image 1
Finished image 2
Finished image 3
Finished image 4
Finished image 5
Finished image 6
Finished image 7
Finished image 8
Finished image 9
Finished image 10
